In [8]:
# Importing all the required libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt, ceil
import argparse

In [9]:
# Constant
DAYSINMONTH=[31,28,31,30,31,30,31,31,30,31,30,31]

### read_all
This function reads all the required files through the directories passed as arguments. It also removes irrelevant data as described in the readme file.

In [10]:
def read_all(catalog_dir, historical_dir, inventory_dir):
    catalog = pd.read_csv(catalog_dir)
    historical_all = pd.read_csv(historical_dir)

    product_names = catalog['Product Name'].values.tolist()
    del catalog

    # removing irrelevant data from historical dataframe.
    condition = (historical_all['productname'].isin(product_names))
    historical = historical_all.loc[condition]
    
    del historical_all
    historical = historical.drop(['orderid', 'customerid', 'barcode'], axis=1)

    historical['ordertime'] = pd.to_datetime(historical['ordertime'])

    # reading Inventory
    inventory_all = pd.read_csv(inventory_dir)

    # removing irrelevant data from inventory
    condition = (inventory_all['Product Name'].isin(product_names))
    inventory = inventory_all.loc[condition]
    #del inventory_all,condition

    # resetting index for all
    historical = historical.reset_index()
    inventory = dict(inventory.values.tolist())

    return historical, inventory, product_names

### poly_regr
Initially this ran the SVR and Polynomial regression models (of  greater than 3 degrees) but now takes the raw information, processes it into model-friendly format and then establishes upper limits and lower limits. It returns the total predicted amount for 6 months, the trained model (originally for debug purposes) and the number of days current inventory will last.

In [11]:
def poly_regr(info, curr_inv,  X_feat = 'timestamp', y_feat = 'qi', qi = True):

    # Create X and y to be fed into model.
    X = []
    y = []
    temp = 0
    quant = 0
    for x in info:
        X.append(int(x[X_feat]))
        y.append(x[y_feat])
        temp=temp+x['count']
        quant = quant + x['quantity']
    X = np.array(X).reshape(-1,1)
    y = np.array(y)
    if len(X) == 0:
        return 0, None, None
    avg_y = sum(y)/len(X)
    avg_q = quant/len(X)
    stddev_y = 0
    stddev_q = 0
    for i in range(len(X)):
        stddev_y = stddev_y + abs(avg_y - y[i])
        stddev_q = stddev_q + abs(avg_q - info[i]['quantity'])
        
    temp = temp/len(X)
    one = ceil((avg_q+2*stddev_q)*6)
    two = (avg_y+2*stddev_q)
    # Create features
    poly = PolynomialFeatures(degree = 1)
    poly_x = poly.fit_transform(X)
    
    # Create model and train
    model = LinearRegression()
    model.fit(poly_x, y)
    total = 0
    qi_avg = 0
    for i in range(1,6):
        if not qi:
            total = total + model.predict(poly.fit_transform(np.array([int(pd.datetime(2019,i,1).timestamp())]).reshape(-1, 1)))[0]
        else:
            qi_pred = max(min(model.predict(poly.fit_transform(np.array([int(pd.datetime(2019,i,1).timestamp())]).reshape(-1, 1)))[0],two),1)
            total = total + qi_pred*temp
            qi_avg += qi_pred
    total = min(ceil(total),one)
    days = int(curr_inv*181/total)
    return total, model, days

### pred_six_months
This function processes the predictions for one product for 6 months. Takes the raw transaction data and extracts the following features:
* date (for readability purposes)
* timestamp (for model purpose)
* quantity 
* count (number of transactions)
* qi (quantity by count)

In [12]:
def pred_six_months(productname, df, curr_inv):

    # narrow down information to what's necessary.
    condition = (df['productname']==productname)
    df = df.loc[condition].reset_index()

    # get raw information from raw data.
    info = []
    size = len(df)
    for i in range(size):
        curr_dt = df.ordertime.iloc[i]
        year = curr_dt.year + int((curr_dt.month+1)/12)
        month = max((curr_dt.month + 1) % 13, 1)
        day = min(DAYSINMONTH[month-1], curr_dt.day)
        up_limit = pd.datetime(year, month, day)
        if up_limit >= df.ordertime.iloc[size-1]:
            break
        
        j = i
        total_quantity = 0
        count = 0
        while df.ordertime[j] < up_limit:
            total_quantity = total_quantity + df.qty.iloc[j]
            j = j+1
            count = count + 1
        info.append({"date":curr_dt,
         "timestamp":curr_dt.timestamp(),
         "quantity":total_quantity,
         "count":count,
         "qi":(total_quantity/count)})
    
    # Feed into polynomial regression model and return value
    
    return poly_regr(info, curr_inv)

### orchestrate()
This function orchestrates the entire process. It takes each product one after the other and runs the pred_six_months function. It also writes the final data into a csv file.

In [13]:
def orchestrate(historical, inventory, product_names):
    data = []
    q2_p1 = False
    leftover = 0
    itr = 0
    for product_name in product_names:
        curr_inv = inventory[product_name]
        pred, model, days = pred_six_months(product_name, historical, curr_inv)
        if pred == 0:
            data.append([product_name, curr_inv, 0, 0])
            continue
        #DEBUG
        print(product_name, "\t", pred)
        
        #days = get_days(model, curr_inv, pred, poly)
        data.append([product_name, curr_inv, days, int(max(pred-curr_inv, 0))])
        if product_name == 'Britannia Bisc Gday Cashew 100g':
            #num = get_reorder_amt(pred)
            #print("num is ", num, "predicted amount is ", pred)
            if pred > 200:
                q2_p1 = True
                leftover = 300-pred
        del model
        itr = itr+1
    df = pd.DataFrame(data,columns=['Product Name','Current Inventory', "Number of days the current inventory will last starting 1st Jan 2019", "Quantity  to purchase for the next six months"])
    
    #print(df)
    df.to_csv(r'output.csv')
    
    print("\n\nAnswer to Q1 has been saved as output.csv\n\n")
    print("The answer to Q2: ")
    if q2_p1:
        print('Yes, the offer is profitable and the total predicted expired units are ',leftover)
    else:
        print("No, the offer isn't profitable. ")
    return df

# The main code snippet

Fill in the catalog, historical and inventory directories and let the code run.

In [14]:
catalog_dir = 'britannia_catalogue.csv'
historical_dir = 'historicaldata.csv'
inventory_dir = 'current_inventory.csv'
historical, inventory, product_names = read_all(catalog_dir = catalog_dir, historical_dir = historical_dir, inventory_dir = inventory_dir)
df = orchestrate(historical, inventory, product_names)
print(df)

Britannia Cake Chocolate 45g 	 630
Britannia Slt Bis Mska Chska 50g 	 614
Britannia Bisc Litle Heart 37g 	 641
Britannia Gd Chocochip 100gm 	 512
Britannia Cake Pineapple 45g 	 519
Britannia Cake Fruit 45g 	 444
Britannia Cake Orange 45g 	 476
Britannia Biscuit Milkbikis 85g 	 437
Britannia Crm Bisc Jim Jam 100g 	 441
Britannia Cakes Choco Muffills Pp 35g 	 406
Britannia Cake Chocolate 130g 	 315
Britannia Crm Bisc Trt Bourbon 60g 	 291
Britannia Cake Butter Sponge 45g 	 260
Britannia Cake Fruit 130g 	 290
Britannia Cakes Strawberry Muffills 35g 	 553
Britannia Crm Bisc Bourbon 120g 	 260
Britannia Crm Bisc Milk Bikis 100g 	 243
Britannia Bisc Digestive 100g 	 215
Britannia Bisc Gday Cashew 100g 	 243
Britannia Bis Mgic Deuce Milk Choco 60g 	 534
Britannia Bisc Good Day Nuts Pp 100g 	 207
Britannia Bis Prm Suji Rusk 200g 	 206
Britannia Crm Bis Orange Treat 75g 	 196
Britannia Crm Bisc Trt Bourbon 150g 	 162
Britannia Sb 50-50 Maska Chaska Pp 120g 	 195
Britannia Bisc Nice Time 73g 	 1